# Dataset Preprocessing


This tutorial contains preprocessing information necessary to prepare the dataset that we will use during our labs.
We will focus on the [Movielens latest small](https://www.kaggle.com/grouplens/movielens-latest-small) dataset. Other datasets can be found on Canvas, in the [Project Resources](https://canvas.maastrichtuniversity.nl/courses/7954/pages/project-resources?module_item_id=203858) module.

#### 1. Preprocessing the Movielens dataset

We download the Movielens dataset and we insert the files in the folder *movielens_dataset_small*. The dataset contains several files CSV. We can start using the [pandas](https://pandas.pydata.org/) library to read the content of the CSV files and use it.
Let us have a look at the first 10 rows to check which contents are provided by the *ratings.csv* and the *movies.csv* files. To do so, we use the *read_csv* method which returns a pandas [Dataframe](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) object containing the information read from the CSV file. Such object provides several methods to manipulate the data in it, and we will use the [head](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.head.html) method, which returns the first *n* rows of the Dataframe. We use the [display](https://ipython.readthedocs.io/en/stable/api/generated/IPython.display.html#IPython.display.display) method, that allows to print information nicely in a jupyter notebook.

In [1]:
movielens_dataset_folder = "./movielens_dataset_small"

import pandas as pd

In [2]:
ratings_df = pd.read_csv(movielens_dataset_folder+"/ratings.csv") 
display(ratings_df.head(10))

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
5,1,70,3.0,964982400
6,1,101,5.0,964980868
7,1,110,4.0,964982176
8,1,151,5.0,964984041
9,1,157,5.0,964984100


In [3]:
movies_df = pd.read_csv(movielens_dataset_folder+"/movies.csv")
display(movies_df.head(10))

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


First, we need to modify some of the columns. The [Lenskit](https://lkpy.readthedocs.io/en/stable/) library, that we will use as recommendation engine, requires that the *ratings_df* Dataframe has the columns *user* and *item*, and we would also need the column *item* in the *movies_df* Dataframe. We will use the [rename](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html) function of the Dataframe class to rename the columns.

In [4]:
ratings_df = ratings_df.rename(columns={'userId': 'user', 'movieId': 'item'})
movies_df = movies_df.rename(columns={'movieId': 'item', 'title' : 'title_original'})
display(ratings_df.head(10))
display(movies_df.head(10))

,user,item,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
5,1,70,3.0,964982400
6,1,101,5.0,964980868
7,1,110,4.0,964982176
8,1,151,5.0,964984041
9,1,157,5.0,964984100


,item,title_original,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


##### 1.1 Split title and year in two different columns

First, we can see that the *title_original* column contains the titles and the year in the same field. It is better to separate such information, that allow us to properly identify a movie, since different movies might have the same title. At the same time, we will replace the title with a lower case versionb of it. This will help us in future title comparisons.

We will use the [apply](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html) method of the Dataframe object. Such method takes as input a function. In the code below, the function is called on each row of the Dataframe on which the *apply* method is called.

Hence, we define a function `compute_title_year` that has a row as input. This function should check if the *title* of this movie contains the year (since for some of the movies the year might not be present). If so, the *year* is extracted from the *title_original*, and a new title is also computed. We will also use some functions of the [Text Sequence Type - str](https://docs.python.org/3/library/stdtypes.html#text-sequence-type-str). You can see in the comments what each instruction do, while for a more detailed reference of each method follow this [link](https://docs.python.org/3/library/stdtypes.html#string-methods).

In [5]:
def compute_title_year(row):
    # read the title of the specified row, and cast it to the str type,
    # then use the strip() method to eliminate the spaces from the beginning and the end of the string
    title_original = str(row['title_original']).strip() 
    
    # select the substring from the fifth-to-last (included) character to the last (excluded)
    year = title_original[-5:-1] 
    
    # use the isdigit method to determine if the substring selected is a number
    if year.isdigit():
        # remove the year from the title
        # select the substring from the beninning to the sixth-to-last (excluded)
        # then use the strip() method to remove spaces at the beginning and at the end of the string
        # finally, use the lower() method to obtain the title in lower case
        title = title_original[:-6].strip().lower()
        
        # return the year and the title
        return int(year), title
    else:
        # nothing to do, return 0 as year and the title
        return 0, title_original.lower()

Now, we use the *apply* method to execute the *compute_title_year* function on all the rows in our dataframe. We assign the resulting columns in tho columns *year* and *title* (*year* is a new column, while *title* will be overwritten).

In [6]:
movies_df[['year', 'title']] = movies_df.apply(compute_title_year, axis=1, result_type ='expand')
display(movies_df.head(10))

,item,title_original,genres,year,title
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,toy story
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995,jumanji
2,3,Grumpier Old Men (1995),Comedy|Romance,1995,grumpier old men
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995,waiting to exhale
4,5,Father of the Bride Part II (1995),Comedy,1995,father of the bride part ii
5,6,Heat (1995),Action|Crime|Thriller,1995,heat
6,7,Sabrina (1995),Comedy|Romance,1995,sabrina
7,8,Tom and Huck (1995),Adventure|Children,1995,tom and huck
8,9,Sudden Death (1995),Action,1995,sudden death
9,10,GoldenEye (1995),Action|Adventure|Thriller,1995,goldeneye


We can see now that we have the two information correctly separated in two different columns

##### 1.2 Eliminate duplicate entries in the Movielens dataset

It is always good to check the information contained in the dataset, even when it is widely used. In particular, we don't want to have repeated entries (different rows of our movies dataset referring to the same movie). We can check this grouping our rows by *title* and *year*, and counting how many rows belong to each group. We use the [groupby](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html) method to group the columns given a specific column value. The method returns a [Groupby](https://pandas.pydata.org/docs/reference/groupby.html) object, which provides methods to aggregate the values of each group. We use the [count](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.GroupBy.count.html) method to check how many rows we have with the same title. The method returns a Dataframe object, in which we will have a row for each movie, and the number of repeated columns *item* and *genres* columns.

Then, we select the movies having a number of entries greater than 1. We will use the [loc](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.loc.html) attribute of the Dataframe object, which allows to access a group of rows and columns by label(s) or a boolean array. In particular, we will determine a Series of boolean: the instruction `count_movies['item'] > 1` evaluate the condition for each row of the *item* column. Hence, *loc* will only return the rows for which such condition is true.

In [7]:
count_movies = movies_df.groupby(['title','year']).count()
sel = count_movies.loc[count_movies['item']> 1]
display(sel)
display(sel.shape)

,,item,title_original,genres
title,year,,,
confessions of a dangerous mind,2002,2,2,2
emma,1996,2,2,2
eros,2004,2,2,2
saturn 3,1980,2,2,2
war of the worlds,2005,2,2,2


(5, 3)

We can check one of the movies to better visualize the problem. We select the rows having a specific title and year, again using the *loc* attribute.

In [8]:
movies_df.loc[(movies_df['title']=="emma") & (movies_df['year']==1996)]

,item,title_original,genres,year,title
650,838,Emma (1996),Comedy|Drama|Romance,1996,emma
5601,26958,Emma (1996),Romance,1996,emma


We can see that the two rows have also different metadata. We will remove the duplicates using the function [drop_duplicates](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html) of the Dataframe object. We specify that we will consider only the column *title* to identify the duplicates, using the *subset* parameter, and we also specify to keep the first duplicate found, using the *keep* parameter. We check again the number of duplicates as we did before.

In [9]:
movies_df = movies_df.drop_duplicates(subset='title', keep='first')

count_movies = movies_df.groupby('title').count()
sel = count_movies.loc[count_movies['item'] > 1]
display(sel)
display(sel.shape)

,item,title_original,genres,year
title,,,,


(0, 4)

We can see that now we don't have any repeated entry.

#### 2. Preprocessing the Wikipedia Movie Plots dataset

In order to implement content-based recommenders, we will need some information nabout the content of our movies. So, we will integrate our dataset with the movies plots. There are different options for this. Several API online provide this information, but they require to perform different calls and each API has some day limits for this. Hence, to save some time, we will use a dataset providing such information. The [Wikipedia Movie Plots](https://www.kaggle.com/jrobischon/wikipedia-movie-plots) dataset provides descriptions of 34,886 movies from around the world. Let's open it and check the provided content.

In [10]:
wiki_plots_dataset_folder = "./wiki_movie_plots_deduped"
plots_df = pd.read_csv(wiki_plots_dataset_folder+"/wiki_movie_plots_deduped.csv") 
display(plots_df.head(10))

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...
5,1903,Alice in Wonderland,American,Cecil Hepworth,May Clark,unknown,https://en.wikipedia.org/wiki/Alice_in_Wonderl...,"Alice follows a large white rabbit down a ""Rab..."
6,1903,The Great Train Robbery,American,Edwin S. Porter,NaN,western,https://en.wikipedia.org/wiki/The_Great_Train_...,The film opens with two bandits breaking into ...
7,1904,The Suburbanite,American,Wallace McCutcheon,NaN,comedy,https://en.wikipedia.org/wiki/The_Suburbanite,The film is about a family who move to the sub...
8,1905,The Little Train Robbery,American,Edwin Stanton Porter,NaN,unknown,https://en.wikipedia.org/wiki/The_Little_Train...,The opening scene shows the interior of the ro...
9,1905,The Night Before Christmas,American,Edwin Stanton Porter,NaN,unknown,https://en.wikipedia.org/wiki/The_Night_Before...,Scenes are introduced using lines of the poem....


In this case, we already have the title and the year in two separate columns. To ensure the comparability with the data in the Movielens dataset, however, we need to compute a lowercase version of the titles in the *plots_df* Dataframe. We can do it by operating directly on the *Title* column. We select the column, obtaining a [Series](https://pandas.pydata.org/docs/reference/api/pandas.Series.html) object. On that, we use the [str](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.html) function, than we use the *strip* method to remove spaces from the beginning and the end of the string. We obtain a new Series object, hence we need to use again the *str* function before we can apply the *lower* function. At the end, we assign the final Series object to the new column *title*.

We will also rename the *Release Year* column anbd the *Plot* column, and the select only the information we are interested in.

In [11]:
plots_df['title'] = plots_df['Title'].str.strip().str.lower()
plots_df = plots_df.rename(columns={'Release Year': 'year', 'Plot' : 'plot'})
plots_df = plots_df[['title', 'year', 'plot']]

display(plots_df.head(10))

,title,year,plot
0,kansas saloon smashers,1901,"A bartender is working at a saloon, serving dr..."
1,love by the light of the moon,1901,"The moon, painted with a smiling face hangs ov..."
2,the martyred presidents,1901,"The film, just over a minute long, is composed..."
3,"terrible teddy, the grizzly king",1901,Lasting just 61 seconds and consisting of two ...
4,jack and the beanstalk,1902,The earliest known adaptation of the classic f...
5,alice in wonderland,1903,"Alice follows a large white rabbit down a ""Rab..."
6,the great train robbery,1903,The film opens with two bandits breaking into ...
7,the suburbanite,1904,The film is about a family who move to the sub...
8,the little train robbery,1905,The opening scene shows the interior of the ro...
9,the night before christmas,1905,Scenes are introduced using lines of the poem....


##### 2.1 Removing duplicate entries from the Wikipedia Movie Plots Dataset

Now, we can check again duplicates. Again, we group by *title* and *year*.

In [12]:
count_movies = plots_df.groupby(['title','year']).count()
sel = count_movies.loc[count_movies['plot'] > 1]
display(sel)
display(sel.shape)

,,plot
title,year,
12,2006,2
12 years a slave,2013,2
123,2002,3
13,2006,2
14,2006,3
...,...,...
woman in gold,2015,2
x-men: days of future past,2014,2
x-men: first class,2011,2


(316, 1)

We can see that there are many duplicates. We will remove them as we did in the previoius case, using the *drop_duplicates* function.

In [13]:
plots_df = plots_df.drop_duplicates(subset=['title','year'], keep='first')

count_movies = plots_df.groupby(['title','year']).count()
sel = count_movies.loc[count_movies['plot'] > 1]
display(sel)
display(sel.shape)

,,plot
title,year,


(0, 1)

We now have no duplicates in our datasets. So now we can merge the information in the two Dataframes.

#### 3. Merging the two datasets

Now, we have all the information to merge the two Dataframes. We use the [merge](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html) function in this case, that works similarly to a database join. We specify the two datasets, than `how='inner'` specifies that we will perform an inner join (hence we will only keep movies for which we have the corresponding plot), and then, in the correct order, the join attibutes for the first Dataframe (`left_on=['title','year']`) and for the second Dataframe (`right_on = ['title','year']`). 

We also use the [shape](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.shape.html) attribute to print the number of movies for which we have the plot. We will also select only the columns *item*, *title*, *year*, *genres*, and *plot*.

In [14]:
movies_df = pd.merge(movies_df, plots_df,  how='inner', left_on=['title','year'], right_on = ['title','year'])
movies_df = movies_df[['item','title','year','genres','plot']]
print("Number of movies: " + str(movies_df.shape[0]))      
display(movies_df.head(10))

Number of movies: 4633


,item,title,year,genres,plot
0,1,toy story,1995,Adventure|Animation|Children|Comedy|Fantasy,In a world where toys are living things who pr...
1,2,jumanji,1995,Adventure|Children|Fantasy,"In 1869, near Brantford, New Hampshire, two br..."
2,3,grumpier old men,1995,Comedy|Romance,The feud between Max (Walter Matthau) and John...
3,4,waiting to exhale,1995,Comedy|Drama|Romance,"""Friends are the People who let you be yoursel..."
4,5,father of the bride part ii,1995,Comedy,The film begins five years after the events of...
5,6,heat,1995,Action|Crime|Thriller,"Neil McCauley, a career criminal, hires Waingr..."
6,7,sabrina,1995,Comedy|Romance,Sabrina Fairchild is the young daughter of the...
7,8,tom and huck,1995,Adventure|Children,The movie opens with Injun Joe (Eric Schweig) ...
8,9,sudden death,1995,Action,Darren McCord (Jean-Claude Van Damme) is a Fre...
9,10,goldeneye,1995,Action|Adventure|Thriller,"In 1986, at Arkhangelsk, MI6 agents James Bond..."


We have now in the movies table a subset of the original movies (only the movies for which we had a plot associated). We focus on such movies, hence we only select the ratings given to movies for which we have the plot. We use the *loc* attribute of the Dataframe object. In particular, we will determine a Series of boolean: The instructon `ratings_df['item'].isin(movies_df['item'])` evaluate the condition for each row of the *item* column. We use then the [isin](https://pandas.pydata.org/docs/reference/api/pandas.Series.isin.html) method to select the rows from our *ratings_df* Dataframe having a *item* that is in the list of *item* of the *movies_df* Dataframe. Hence, *loc* will only return the rows for which such condition is true.

In [15]:
print(ratings_df.shape[0])
filtered_ratings_df = ratings_df.loc[ratings_df['item'].isin(movies_df['item'])]
print(filtered_ratings_df.shape[0])

100836
64438


#### 4. Saving our datasets

We have now all the information we need. Let's save our dataset using the [to_csv](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html) function on the Daframe class. 

In [16]:
preprocessed_dataset_folder = "../preprocessed_dataset"
filtered_ratings_df.to_csv(preprocessed_dataset_folder+"/ratings.csv", index=False) 
movies_df.to_csv(preprocessed_dataset_folder+"/movies.csv", index=False) 